# Вариант 2. Нахождение расстояния по фокусному расстоянию

In [ ]:
import utils
import torch
import math
import numpy as np 
from PIL import Image
import cv2
import os
from pathlib import *
import shutil
import pandas as PD
import pillow_heif
import matplotlib.pyplot as plt
%matplotlib inline
PD.options.display.expand_frame_repr = False

In [ ]:
display = utils.notebook_init() 
print('cuda доступна', torch.cuda.is_available())
print('под каким номером CUDA:',torch.cuda.current_device())
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #без этого ошибка будет

In [ ]:
#функция крутит картинку в зависимости от того, что нашла в характеристиках файла. 
def exif_transpose(img):
    if not img:
        return img
    wh = img.size
    if wh[0] > wh[1]:
        return img
    exif_orientation_tag = 274

    # Check for EXIF data (only present on some files)
    if hasattr(img, "_getexif") and isinstance(img._getexif(), dict) and exif_orientation_tag in img._getexif():
        exif_data = img._getexif()
        orientation = exif_data[exif_orientation_tag]

        # Handle EXIF Orientation
        if orientation == 1:
            # Normal image - nothing to do!
            pass
        elif orientation == 2:
            # Mirrored left to right
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 3:
            # Rotated 180 degrees
            img = img.rotate(180)
        elif orientation == 4:
            # Mirrored top to bottom
            img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 5:
            # Mirrored along top-left diagonal
            img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 6:
            # Rotated 90 degrees
            #print('повернули')
            img = img.rotate(-90, expand=True)
        elif orientation == 7:
            # Mirrored along top-right diagonal
            img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 8:
            # Rotated 270 degrees
            img = img.rotate(90, expand=True)

    return img

In [ ]:
#model = torch.hub.load('ultralytics/yolov5', 'custom',  path='runs\\train\\bw\\exp4\\weights\\last.pt') #работает, но загружает каждый раз ЛУЧШЕЕ
model = torch.hub.load('ultralytics/yolov5', 'custom',  path='my_model\\exp4\\weights\\best.pt') #работает, но загружает каждый раз

In [ ]:
def get_grayscale2(image):  #уменьшает изображение в 3 раза, что и хотелось бы сделать
    #https://habr.com/ru/post/163663/
    #convert image -colorspace gray image
    img = Image.open(image).convert('L')
    img.save(image)

In [ ]:
from exif import Image as exif
def  get_focus_from_exif(f):
    with open(f, "rb") as palm_1_file:
        palm_1_image = exif(palm_1_file)
    if palm_1_image.has_exif:
        focal = palm_1_image.get('focal_length_in_35mm_film', 'Unknown')
        digital_zoom = palm_1_image.get('digital_zoom_ratio', 'Unknown')  #поиграем с зумом....а нет. зум начинает менять фокусное расстояние и он не нужен, но пусть будет пока.
    else:
        focal = 0
        digital_zoom = 1
    
    return focal, digital_zoom

In [ ]:
#конвертируем heic_to_jpg. на входе - исходный файл и путь к новому файлу, возвращает путь к новому файлу
def conv_heic_to_jpg(file, new_name):

    heif_file = pillow_heif.read_heif(file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
    )

    ex = heif_file.info['exif']
    image.save(new_name, format("jpeg"), exif=ex)
    image.close()
    return new_name

In [ ]:
#распознавание файла и получение из него данных по централььному номеру
def res(file, show_pandas = 0):
    if '.heic' in file:
        temp_file = 'tmp.jpg'  #нужен временный файл, чтобы не портить датасет
        file = conv_heic_to_jpg(work_file,temp_file)
    
    im = Image.open(file)
    (width, height) = im.size
    
    results = model(file)
    pd = results.pandas().xyxy[0]
 
    pd = pd.loc[pd['name'] == 'znak']                   #таблица со знаками
    
    #все номера нам не надо. Нам надо только тот, что ближе к центру

    #ОБРАБАТЫВАЕМ ЗНАКИ
    if show_pandas == 1:
        print('Знаки\n',pd)
    
    pd = pd.assign(to_centre_x = abs(width/2 - (pd.xmin + (pd.xmax-pd.xmin)/2)))     #считаем расстояние до центра
    pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)                         #считаем координаты центра 
    
    pd = pd.assign(width = pd.xmax - pd.xmin)                                        #ширина
    pd = pd.assign(height = pd.ymax - pd.ymin)                                       #высота
    pd = pd.assign(s2 = pd.width * pd.height)                                        #площадь
    
    pd = pd.sort_values(['to_centre_x'] )                                            #сортируем по близости к центру. Нам надо самый близкий номер
    
    
    d = dict()   #в этом словаре будут все результаты
    for index, row in pd.iterrows():
        d ['width']  = row['width']
        d ['height']  = row['height']
        d ['s2']  = row['s2']
        d ['xmin']  = row['xmin']
        d ['ymin']  = row['ymin']
        d ['xmax']  = row['xmax']
        d ['ymax']  = row['ymax']
        break
    #Сейчас в d - все характеристики знака. 
    
    d['im_width'] = width
    d['im_height'] = height
    
    
    return d    
        
             

In [ ]:
#процедура вырезает знак, просто чтобы посмотреть, что там нашлось. Нужно было для отладки
#на входе - файл, и словарь с данными о знаке
#ВНИМАНИЕ! ИСПОРТИТ полученный файл
def crop_znak(temp_file,znak):
    if len(znak) == 3:
        print('не смог кроопнуть', temp_file)
        return
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((xmin, ymin,xmax, ymax))
    img.save(temp_file)


In [ ]:
#вырезаем область вокруг знака, чтобы подать ее потом снова в сеть для более точного распознавания. 
def mini_pic(temp_file,znak):
    if len(znak) == 3:
        print('не смог кроопнуть', temp_file)
        return
    if znak['width'] >400: #режем только если знак очень мелкий
        return
    
    img = Image.open(temp_file)
    xmin  = znak['xmin']
    ymin = znak['ymin']
    xmax  = znak['xmax']
    ymax  = znak['ymax']
    centre_x = xmin + (xmax - xmin)/2
    centre_y = ymin + (ymax - ymin)/2
    #print((xmin, ymin,xmax, ymax))
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
    #print('минипик',temp_file,centre_x,centre_y, znak['width'])
    
#пытаемся вырезать область вокруг знака, чтобы подать ее потом снова в сеть для более точного распознавания.  
#это если знак на фото мелкий и она вообще ничего не нашла. делаем кроп по центру 960х960
def zoom960(temp_file):
    
    img = Image.open(temp_file)
    w,h = img.size
    centre_x = w//2
    centre_y = h//2
    img = img.crop((centre_x-480, centre_y - 480,centre_x+480, centre_y + 480))
    img.save(temp_file)
 

In [ ]:
#Здесь мы корректируем знак: в зависимости от соотношения между найденными сторонами находим угол и по нему считаем во что превратилось 520
from math import atan, sin, cos, tan,degrees
def correct_znak2(znak):
    
    w_znak = 520 #ширина знака в мм
    h_znak = 112
    if len(znak) == 3: #похоже, что не рааспознался
        return w_znak
    #сначала ищем угол наклона знака по соотношению сторон
    HW  = 112/520 #стандартное соотношение между сторонами
    hw = znak['height']/znak['width']#найденное соотнощение между сторонами
    tan_alfa = -(HW - hw) / (1 - HW * hw)
    alfa_rad = abs(math.atan(tan_alfa)) #нашли угол наклона знака в радианах
    print('приблизительно повернуто',degrees(alfa_rad))
    
    new_AB = w_znak * cos(alfa_rad) + h_znak * sin(alfa_rad)
    #new_w = (CD-AB/tan(alfa_rad)) / (sin(alfa_rad) - cos(alfa_rad)*cos(alfa_rad)/sin(alfa_rad))
    
    print('корректировка длины знака','520', '---->',new_AB)
    
    return new_AB


#старая версия, она корреткирует не 520, а найденное значение ширины уменьшает.
# def correct_znak(znak):
#     if len(znak) == 3: #похоже, что не рааспознался
#         return znak
#     #сначала ищем угол наклона знака по соотношению сторон
#     HW  = 112/520 #стандартное соотношение между сторонами
#     hw = znak['height']/znak['width']#найденное соотнощение между сторонами
#     tan_alfa = -(HW - hw) / (1 - HW * hw)
#     alfa_rad = math.atan(tan_alfa) #нашли угол наклона знака в радианах
#     print('приблизительно повернуто',degrees(alfa_rad))
    
#     CD = znak['height']
#     AB = znak['width']
#     new_w = (CD-AB/tan(alfa_rad)) / (sin(alfa_rad) - cos(alfa_rad)*cos(alfa_rad)/sin(alfa_rad))
    
#     print('корректировка длины знака',znak['width'], '---->',new_w)
#     znak['width'] = new_w
#     return znak
    

In [101]:
def file_recognition(f):
    w_znak = 520 #ширина знака в мм
    #w = 4032    #разрешение матрицы/фото в пикселях
    
    
    work_file = f
    temp_file = 'temp.jpg'
    if '.heic' in f:
        
        work_file = conv_heic_to_jpg(work_file,temp_file)
        print('Сконвертирован в',work_file)
    focus, digital_zoom = get_focus_from_exif(work_file)
    #print('focus, digital_zoom',focus, digital_zoom)
     
    img = Image.open(work_file)
    img = exif_transpose(img)
    w,h = img.size
    img.save(temp_file)
    
    img.close()

    #get_grayscale2(temp_file)  #пытался работать с серым. не очень.
    
    znak1 = res(temp_file)
    znak = znak1
    
    #Заходим еще 1 раз, если ничего не нашли. просто на удачу
    if len(znak) == 3:
        zoom960(temp_file)
        znak1 = res(temp_file)
        znak = znak1
        
    
    
    #ЭКСПЕРИМЕНТ
    if len(znak1) <= 3:
        print('ФАЙЛ НЕ РАСПОЗНАН', f)
        not_recognize.append(f) 
        return 0.0
    
    #пробуем вырезать область,чтобы знак был бллиже. количество пикселей от этого не меняется
    print('ширина до мини пика', znak1['width'])

    mini_pic(temp_file,znak1)
    #вырезали и еще раз на распознавание
    znak = res(temp_file)
    if len(znak) != 3:
        print('ширина после мини пика', znak['width'], 'уточнение', znak1['width']-znak['width'])
    else: #почему то после зума, знак не распознался. Возвращаем то, что было, это всяко лучше чем ноль
        znak = znak1
        
    #....конец эксперимента.
    
    
    #crop_znak(temp_file,znak)
    
    
    #уточнение полученнного расстояние
    w_znak = correct_znak2(znak)
    
    w_matrix = 35# 34.974 #ширина матрицы в мм. Вроде бы именнь это классическая ширина, но ее в разговоре просто округляют до 35 мм.
    #print('ширина кадра',w)
    pixel_in_mm = w/w_matrix
    
    if len(znak) == 3:
        print('ФАЙЛ НЕ РАСПОЗНАН', f)
        not_recognize.append(f)
        d = 0.0
    #формируем строку с параметрами знака для записи в csv
    else:
        
        d = (1 + w_znak /(znak['width']/pixel_in_mm))* focus/1000  #ну вот тут я заморочился. не мог решить до чего считать расстояние - до оптического центра или до матрицы. Стал считать до матрицы. Объектив может гулять от матрицы на много см.
        
    return d

In [103]:
train_csv = '..\\data_set\\train.csv'  #данные датасета если прогонять на тренировочном датасете, то будет считаться ошибка.
keys = ['width', 'height', 's2', 'xmin', 'ymin', 'xmax', 'ymax','im_width', 'im_height']
pic_data = '..\\data_set\\train'  #здесь лежит датасет
pic_data_test  = pic_data
#ДЛЯ ПРОГОНА НА СВОЕМ ДАТАСЕТЕ НАДО ИЗМЕНИТЬ ЭТОТ ПУТЬ
pic_data_test = '..\\data_set\\test'  #здесь лежит тестовы датасет




test = '..\\data_set\\sample_solution.csv'  #сюда запишется резултат

#сначала прочитаем все данные нам растояния из тренировочного датасета
dist = dict()  #здесь будем храть все расстояния добытые из файла. В виде словаря
with open(train_csv, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.replace('\n','')
        key, d = line.split(';')
        dist[key] = d
        
    


new_f = 'image_name;distance'
not_recognize = []  #здесь будут не распознанные
n = 0
abs_mistake, otn_mistake, vsego = 0, 0, 0  #для подсчета ошибки
for f in os.listdir(pic_data_test):
    n +=1
#     if n<= 380:
#         continue
#     if n>25:
#         break
    
    
    work_file = os.path.join(pic_data_test, f)
    print(n,'работаем  с файлом: ',work_file)

    itog = file_recognition(work_file)
    
    #получаем истинное значение, считаем ошибку
    if f in dist:
        y = float(dist[f])
        mistake = (y - itog) / y #извините, что не пользуюсь вашей ошибкой, но мне в процентах понятнее, а так как я не тренирую на ошибке, то сделал так.
        print('ошибка',mistake, 'ист', y, 'предск',itog)
        otn_mistake += mistake  #это нужно, чтобы смотреть сваливается в плюс или в минус
        abs_mistake += abs(mistake)
        vsego +=1
    
    
    st = f+';' + str(itog)
    print('Записан результат: ',st)
    print('\n\n')    
    new_f = new_f +'\n'+st
    
#print(new_f)
with open(test, 'w', encoding = 'utf-8') as file:
    file.write(new_f)
print('не распознано файлов', len(not_recognize))
print('не распознанные', *not_recognize)
print('сохранено в ',test)
if vsego != 0:
    print('Абс ошибка', round(abs_mistake/vsego*100, 2))
    print('Относительная ошибка', round(otn_mistake/vsego*100, 2))
else:
    print('Ошибка не посчитана. Истинных данных не обнаружено. Датасет тестовый?')

1 работаем  с файлом:  ..\data_set\test\img_1597.jpg
ширина до мини пика 276.047119140625
ширина после мини пика 280.30010986328125 уточнение -4.25299072265625
приблизительно повернуто 0.7770951186299437
корректировка длины знака 520 ----> 521.4711680611861
Записан результат:  img_1597.jpg;3.660413254561868



2 работаем  с файлом:  ..\data_set\test\img_1598.jpg
ширина до мини пика 386.4049072265625
ширина после мини пика 400.44805908203125 уточнение -14.04315185546875
приблизительно повернуто 0.8346125227730413
корректировка длины знака 520 ----> 521.5762484159845
Записан результат:  img_1598.jpg;2.5677800618122624



3 работаем  с файлом:  ..\data_set\test\img_1599.jpg
ширина до мини пика 531.33447265625
ширина после мини пика 531.33447265625 уточнение 0.0
приблизительно повернуто 3.0992607395420206
корректировка длины знака 520 ----> 525.2948154433794
Записан результат:  img_1599.jpg;1.9531389473215344



4 работаем  с файлом:  ..\data_set\test\img_1602.jpg
ширина до мини пика 384.2

In [99]:
print(file_recognition('Y:\\255_1.jpg'))

print('\n')
print(file_recognition('Y:\\255_2.jpg'))


print('\n')

print(file_recognition('Y:\\255_3.jpg'))


print('\n')

ширина до мини пика 621.3426513671875
ширина после мини пика 621.3426513671875 уточнение 0.0
приблизительно повернуто 0.8046672575805827
корректировка длины знака 520 ----> 521.5216060529401
2.596262762233852


ширина до мини пика 741.8291015625
ширина после мини пика 741.8291015625 уточнение 0.0
приблизительно повернуто 0.095661303130916
корректировка длины знака 520 ----> 520.1862708696381
2.4954518209828622


ширина до мини пика 1238.895751953125
ширина после мини пика 1238.895751953125 уточнение 0.0
приблизительно повернуто 0.6189050271827519
корректировка длины знака 520 ----> 521.1794557392569
2.532044576790743




In [90]:
print(file_recognition('Y:\\421_1.jpg'))
print('\n')
print(file_recognition('Y:\\421_2.jpg'))
print('\n')

print(file_recognition('Y:\\421_3.jpg'))
print('\n')

4.225213614729331


4.175909709691168


4.30194002688896




In [91]:
print(file_recognition('Y:\\490_1.jpg'))
print('\n')
print(file_recognition('Y:\\490_2.jpg'))
print('\n')

print(file_recognition('Y:\\490_3.jpg'))
print('\n')

4.923146672370446


4.690990338609638


4.974977451052313




# Далее эксперименты, можно не смотреть

In [ ]:
def get_grayscale(image):  #это не уменьшает почему то размер изображения
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
n = 0
pic_data = '..\\avto\\train_data3\\images\\train\\'
for f in os.listdir(pic_data):
    n +=1
    work_file = os.path.join(pic_data, f)
    print(n,'работаем  с файлом: ',work_file)
    #img = cv2.imread(work_file)
    img = get_grayscale2(work_file)
    #cv2.imwrite(work_file,img)
print('ОК')

In [ ]:
d = file_recognition('..\\avto\\start_dataset\\test\\img_2717.jpg')
print(d)

In [ ]:
img = Image.open('..\\avto\\start_dataset\\test\\img_2717.jpg')
img = img.rotate(-90, expand=True)
img.save('Y:\\img_2717.jpg')

In [ ]:
from exif import Image as exif

In [ ]:
#не распознанные img_2674.heic img_2715.jpg img_2717.jpg

In [ ]:

get_focus_from_exif("..\\avto\\start_dataset\\test\\img_1602.jpg")

In [ ]:
with open('Y:\\421_2.jpg', "rb") as palm_2_file:
    palm_2_image = exif(palm_2_file)
with open('Y:\\421_1.jpg', "rb") as palm_1_file:
    palm_1_image = exif(palm_1_file)

images = [palm_1_image, palm_2_image]

In [ ]:
if palm_1_image.has_exif:
    status = f"contains EXIF (version {palm_1_image.exif_version}) information."
else:
    status = "does not contain any EXIF information."
print(f"Image {status}")

In [ ]:
image_members = []

for image in images:
    image_members.append(dir(image))

for index, image_member_list in enumerate(image_members):
    print(f"Image {index} contains {len(image_member_list)} members:")
    print(f"{image_member_list}\n")

In [ ]:
palm_1_image.get('focal_length_in_35mm_film', 'Unknown')

In [ ]:
for i in image_member_list:
    print(i,palm_1_image.get(i, 'Unknown'))

In [ ]:
for index, image in enumerate(images):
    print(f"Lens and OS - Image {index}")
    print("---------------------")
    print(f"focal_length_in_35mm_film: {image.get('focal_length_in_35mm_film', 'Unknown')}")
    print(f"Lens model: {image.get('lens_model', 'Unknown')}")
    print(f"Lens specification: {image.get('lens_specification', 'Unknown')}")
    print(f"OS version: {image.get('software', 'Unknown')}\n")

In [ ]:
for i in image_member_list:
    print(i)

In [ ]:
import exifread
f = open('..\\avto\\start_dataset\\test\\img_2674.heic', 'rb')
#f = open('..\\avto\\start_dataset\\test\\img_1598.jpg', 'rb')

tags = exifread.process_file(f,details=False)

In [ ]:
tags['EXIF FocalLengthIn35mmFilm']

In [ ]:
for tag in tags.keys():
    print ("Key: %s, value %s" % (tag, tags[tag]))

In [ ]:
decodeImage('..\avto\start_dataset\test\img_2674.heic')

In [ ]:
heif_file = pillow_heif.open_heif('..\\avto\\start_dataset\\test\\img_2674.heic')
print("image mode:", heif_file.mode)
print("image data length:", len(heif_file.data))
print("image data stride:", heif_file.stride)
heif_file.convert_to("RGB;16")  # convert 10 bit image to RGB 16 bit.
print("image mode:", heif_file.info.keys())

In [ ]:
print(heif_file.info['exif'])

In [ ]:
 heif_file = pillow_heif.read_heif(file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,

In [ ]:
https://pillow-heif.readthedocs.io/en/latest/reference/API.html